# Oasis Reinsurance Test Tool v0.6

This notebook allows example reinsurance structures to be input in OED format and ran against the development version of the Oasis finanical engine. 

## Included
* Basic direct insurance modelling (not the complete engine included in Flamingo)
    * Buildings, Other buildings contents and BI
    * Blanket limit
    * Blanket deductible
    * Site limit
    * Site deductible 
* FAC treaties
* Quota share
* Surplus share
* Per-risk
* Cat XL treaties

## Not included
* Attachement basis
* Aggregate terms
* Multiple portfolios in a single set of input files

## Test cases

A selection of test cases can be found in the examples folder.

* simple_loc_FAC: Single location level fac.
* simple_pol_FAC: Single policy level fac.
* simple_acc_FAC: Single account level fac.
* multiple_FAC: Multiple facs at sameinuring level.
* simple_QS: Single quota share with no risk limits.
* loc_SS: Single surplus share at location level.
* pol_SS: Single surplus share at location level.
* acc_SS: Single surplus share at location level.
* multiple_SS: Multiple surplus shares at same inuring level.
* loc_limit_QS: Single quota share with location level risk limits.
* pol_limit_QS: Single quota share with policy level risk limits.
* acc_limit_QS: Single quota share with account level risk limits.
* multiple_QS_1:  Two quota shares at same inuring level.
* multiple_QS_2:  Two quota shares at different inuring levels.
* simple_CAT_XL: Single cat XL.
* multiple_CAT_XL: Two cat XLs at different inuring levels.


## Input files 

Files must be named the following:
* account.csv
* location.csv
* ri_info.csv
* ri_scope.csv


## Validation Rules
* Risk levels cannot be mixed in a single reinsurance scope
* Values in the scope file must link to rows in ACC/LOC exposure file set.
* QS always has non-specific scope
* SS always has specific scope
* Reinsurance types cannot be combined in an inuring layer

In [ ]:
# Step 1 - notebook and Python setup
%config IPCompleter.greedy=True

# Standard Python libraries
import io
import json
import os

# 3rd party Python libraries
import jupyter_helper
import pandas as pd
import six
    
# Oasis imports
import oasislmf._data as static_data 
from oasislmf.model_preparation import oed
from oasislmf.model_preparation.reinsurance_layer import (
    create_xref_description,
    generate_files_for_reinsurance,
)
from oasislmf.utils.deterministic_loss import (
    generate_oasis_files,
    generate_losses,
)
import reinsurance_tester

In [ ]:
jupyter_helper.file_uploader('examples/uploaded')

In [ ]:
# Step 2 - load the OED for a worked example, in this case FM Test Case 23.
# Note that only the currently used fields are shown unless show_all is set to True. 
oed_dir = os.path.abspath('examples_old/loc_SS')

# Account file
oed_account_file = os.path.join(oed_dir, "account.csv")
if not os.path.exists(oed_account_file):
    print("Path does not exist: {}".format(oed_account_file))
    exit(1)
account_df = pd.read_csv(oed_account_file)

# Location file
oed_location_file = os.path.join(oed_dir, "location.csv")
if not os.path.exists(oed_location_file):
    print("Path does not exist: {}".format(oed_location_file))
    exit(1)
location_df = pd.read_csv(oed_location_file)

(
    ri_info_df, 
    ri_scope_df, 
    do_reinsurance
) = oed.load_oed_dfs(oed_dir, show_all=False)

In [ ]:
# Step 3 - view/edit the account data. 
account_grid = jupyter_helper.show_df(account_df)
account_grid

In [ ]:
# Step 4 - view/edit the location data.
location_grid = jupyter_helper.show_df(location_df)
location_grid

In [ ]:
# Step 5 - view/edit the reinsurance info data.
ri_info_grid = jupyter_helper.show_df(ri_info_df)
ri_info_grid

In [ ]:
# Step 6 - view/edit the reinsurance scope data.
ri_scope_grid = jupyter_helper.show_df(ri_scope_df)
ri_scope_grid

In [ ]:
# Step 7 - Generate the direct Oasis files in `run_reinsurance/` via the MDK
data_fp = static_data.__path__._path[0]
direct_oasis_files_dir = os.path.abspath('run_reinsurance')
oasis_files =  generate_oasis_files(
    direct_oasis_files_dir,
    oed_location_file,
    os.path.join(data_fp, 'MappingMapToOED_CanLocA.xslt'),
    oed_account_file,
    os.path.join(data_fp, 'MappingMapToOED_CanAccA.xslt')
)

In [ ]:
# Step 8 - Prepare the input data to generate the RI input files
xref_descriptions = create_xref_description(account_df, location_df)
items = pd.read_csv(os.path.join(direct_oasis_files_dir, 'items.csv'))
coverages = pd.read_csv(os.path.join(direct_oasis_files_dir, 'coverages.csv'))
gulsummaryxrefs = pd.read_csv(os.path.join(direct_oasis_files_dir, 'gulsummaryxref.csv'))
fmsummaryxrefs = pd.read_csv(os.path.join(direct_oasis_files_dir, 'fmsummaryxref.csv'))
fm_xrefs = pd.read_csv(os.path.join(direct_oasis_files_dir, 'fm_xref.csv'))

In [ ]:
# Step 9 - Generate the RI input files
ri_layers = generate_files_for_reinsurance(
    items,
    coverages,
    fm_xrefs,
    xref_descriptions,
    ri_info_df,
    ri_scope_df,
    direct_oasis_files_dir,
    gulsummaryxrefs,
    fmsummaryxrefs    
)

In [ ]:
# Step 10 - pick up any edits in the grid before running the analysis
account_df = account_grid.get_changed_df()
location_df = location_grid.get_changed_df()
ri_info_df = ri_info_grid.get_changed_df()
ri_scope_df = ri_scope_grid.get_changed_df()

In [ ]:
# Step 11 - run the OED data though the Oasis Financial Module and output the losses by item at each inuring level.
net_losses = reinsurance_tester.run_test('run_reinsurance', account_df, location_df, ri_info_df, ri_scope_df, loss_factor=1.0, do_reinsurance=do_reinsurance)
print("Ran {} inuring layers".format(len(net_losses) - 1))
print("Losses for:")
for key in net_losses.keys():
    print("\t{}".format(key))

In [ ]:
# Step 12 - view the direct losses.
key = 'Direct'
net_losses[key]

In [ ]:
# Step 13 - view the losses for the first inuring layer.
key = 'Inuring_priority:1 - Risk_level:LOC'
net_losses[key]

## Optional Steps


In [ ]:
#Automated testing on reinsurance examples
!python3.6 -m pytest -sp no:flaky -v tests/test_reinsurance.py

In [ ]:
# Write outputs to CSV for download.
for (description, net_loss) in net_losses.items():
    filename = "output_{}.csv".format(description.replace(' ', '_'))
    net_loss.to_csv(filename, index=False)